<a href="https://colab.research.google.com/github/c7934597/Python_Basic_Practice/blob/master/TensorFlowCh2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
'''
讀取影像檔 & 寫入 TFRecord 檔
'''

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [0]:
'''
怎麼用 Python 讀取影像檔，畢竟要寫檔案之前，要先把原始檔案讀取進來。

Python 大概提供幾種讀取影像檔的方法：

1. PIL.Image.open
2. scipy.ndimage.imread
3. scipy.misc.imread
4. skimage.io.imread
5. cv2.imread
6. matplotlib.image.imread

我個人是比較喜歡 5，也就是用 opencv 來幫忙，以下簡單示範讀檔用法
'''

import cv2

# Loads a color image
I1 = cv2.imread('/home/sample.jpg')

# Loads image in grayscale mode
I2 = cv2.imread('/home/sample.jpg', 0)

# Second argument is a flag which specifies the way image should be read.
# cv2.IMREAD_COLOR : Loads a color image. Any transparency of image will be neglected. It is the default flag.
# cv2.IMREAD_GRAYSCALE : Loads image in grayscale mode
# cv2.IMREAD_UNCHANGED : Loads image as such including alpha channel
# Note Instead of these three flags, you can simply pass integers 1, 0 or -1 respectively.

In [0]:
'''
一個資料夾內，包含了 0 ～ 9 個數字，每個資料夾內又有好幾千張圖檔。
這時候，我們得先設計一個函數，把所有的檔案名稱取回來，函數如下


簡單來說，os.walk 會傳回一個由三個元素的 tuple 所組成的串列，
tuple 裡面的值分別是(資料夾名稱、下一層資料夾串列、本資料夾內所有的檔案串列)，
由這些資料組合出所有的往下的樹狀目錄的內容。

從這個 tuple 的第一個參數可以知道目前在處理的資料夾是哪一個；
而第二個參數用來了解在此資料夾中還有幾個下層資料夾，分別叫什麼名字；
第三個參數就是此資料夾中的所有檔案名稱。
'''

def get_File(file_dir):
    # The images in each subfolder
    images = []
    # The subfolders
    subfolders = []

    # Using "os.walk" function to grab all the files in each folder
    for dirPath, dirNames, fileNames in os.walk(file_dir):
        for name in fileNames:
            images.append(os.path.join(dirPath, name))

        for name in dirNames:
            subfolders.append(os.path.join(dirPath, name))

    # To record the labels of the image dataset
    labels = []
    count = 0
    for a_folder in subfolders:
        n_img = len(os.listdir(a_folder))
        labels = np.append(labels, n_img * [count])
        count+=1

    subfolders = np.array([images, labels])
    subfolders = subfolders.transpose()

    '''
    # 無法樹狀目錄無法轉成list時debug
    print(subfolders)

    try:
      image_list = list(subfolders[:,0])
      label_list = list(subfolders[:,1])
      label_list = [int(float(i)) for i in label_list]
    except:
      print('Skip!\n')
    '''
    image_list = list(subfolders[:,0])
    label_list = list(subfolders[:,1])
    label_list = [int(float(i)) for i in label_list]
    
    return image_list, label_list

In [0]:
'''
當我們把所有資料夾內的檔案名稱取回來之後，就可以把檔案轉化成 TensorFlow 專屬的格式：

—— TFRecords檔案格式。

這種檔案格式會把資料轉換成二進位的資料，在讀取時要搭配 TensorFlow 的檔案隊列(或稱佇列，Queue)，才能把資料再讀取出來。

寫入TFRecord 檔案的標準做法是：

step 1. 把所有資料轉換成「tf.train.Feature」格式。

step 2. 把所有的「tf.train.Feature」包裝成「tf.train.Features」格式。

step 3. 把所有的「tf.train.Features」組合成「tf.train.Example」格式。

step 4. 利用「tf.python_io.TFRecordWriter」將「tf.train.Example」寫入成 TFRecord 檔案。

程式碼如下
'''

# 轉Int64資料為 tf.train.Feature 格式
def int64_feature(value):
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

# 轉Bytes資料為 tf.train.Feature 格式
def bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def convert_to_TFRecord(images, labels, filename):
    n_samples = len(labels)
    TFWriter = tf.python_io.TFRecordWriter(filename)

    print('\nTransform start...')
    for i in np.arange(0, n_samples):
        try:
            image = cv2.imread(images[i], 0)

            if image is None:
                print('Error image:' + images[i])
            else:
                image_raw = image.tostring()

            label = int(labels[i])
            
            # 將 tf.train.Feature 合併成 tf.train.Features
            ftrs = tf.train.Features(
                    feature={'Label': int64_feature(label),
                             'image_raw': bytes_feature(image_raw)}
                   )
        
            # 將 tf.train.Features 轉成 tf.train.Example
            example = tf.train.Example(features=ftrs)

            # 將 tf.train.Example 寫成 tfRecord 格式
            TFWriter.write(example.SerializeToString())
        except IOError as e:
            print('Skip!\n')

    TFWriter.close()
    print('Transform done!')

In [0]:
'''
完成寫入 TFRecord 檔案的程式之後，現在可以繼續撰寫主程式來轉檔了!
''' 

import os
import cv2
import numpy as np
import tensorflow as tf

def main():
    # 資料集的位置
    train_dataset_dir = '/home/'
    
    # 取回所有檔案路徑
    images, labels = get_File(train_dataset_dir)
    
    # 開始寫入 TRRecord 檔案
    convert_to_TFRecord(images, labels, '/content/sample_data/Train.tfrecords')

if __name__ == '__main__':
    main()